<a href="https://colab.research.google.com/github/ArulrajGopal/Spark-Guide/blob/main/GogCol/14_groupBy_agg_pivot_cube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

from pyspark.sql import SparkSession

#create spark session
spark= SparkSession.builder.appName('mysparksession').getOrCreate()

#create spark context
sc = spark.sparkContext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=23687fa245c2bdb9e907aa65f11278aa215765305cb71309cbb10194d4a2301f
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql.functions import col,sum,avg,max


In [6]:
simpleData = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]

schema = ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data=simpleData, schema = schema)
df.printSchema()
df.show(truncate=False)


root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|James        |Sales     |NY   |90000 |34 |10000|
|Michael      |Sales     |NY   |86000 |56 |20000|
|Robert       |Sales     |CA   |81000 |30 |23000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|Raman        |Finance   |CA   |99000 |40 |24000|
|Scott        |Finance   |NY   |83000 |36 |19000|
|Jen          |Finance   |NY   |79000 |53 |15000|
|Jeff         |Marketing |CA   |80000 |25 |18000|
|Kumar        |Marketing |NY   |91000 |50 |21000|
+-------------+----------+-----+------+---+-----+



In [7]:
df.groupBy("department").sum("salary").show(truncate=False)

+----------+-----------+
|department|sum(salary)|
+----------+-----------+
|Sales     |257000     |
|Finance   |351000     |
|Marketing |171000     |
+----------+-----------+



In [8]:
df.groupBy("department","state") \
    .sum("salary","bonus") \
   .show(truncate=False)

+----------+-----+-----------+----------+
|department|state|sum(salary)|sum(bonus)|
+----------+-----+-----------+----------+
|Sales     |CA   |81000      |23000     |
|Finance   |CA   |189000     |47000     |
|Sales     |NY   |176000     |30000     |
|Finance   |NY   |162000     |34000     |
|Marketing |NY   |91000      |21000     |
|Marketing |CA   |80000      |18000     |
+----------+-----+-----------+----------+



In [9]:
df.groupBy("department") \
    .agg(sum("salary").alias("sum_salary"), \
      avg("salary").alias("avg_salary"), \
      sum("bonus").alias("sum_bonus"), \
      max("bonus").alias("max_bonus")) \
    .where(col("sum_bonus") >= 50000) \
    .show(truncate=False)

+----------+----------+-----------------+---------+---------+
|department|sum_salary|avg_salary       |sum_bonus|max_bonus|
+----------+----------+-----------------+---------+---------+
|Sales     |257000    |85666.66666666667|53000    |23000    |
|Finance   |351000    |87750.0          |81000    |24000    |
+----------+----------+-----------------+---------+---------+



In [ ]:
# from pyspark.sql.functions import expr

# data = [("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"), \
#       ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"), \
#       ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"), \
#       ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico")]

# columns= ["Product","Amount","Country"]
# df = spark.createDataFrame(data = data, schema = columns)
# df.printSchema()
# df.show(truncate=False)

# pivotDF = df.groupBy("Product").pivot("Country").sum("Amount")
# pivotDF.printSchema()
# pivotDF.show(truncate=False)

# pivotDF = df.groupBy("Product","Country") \
#       .sum("Amount") \
#       .groupBy("Product") \
#       .pivot("Country") \
#       .sum("sum(Amount)")
# pivotDF.printSchema()
# pivotDF.show(truncate=False)

# """ unpivot """
# unpivotExpr = "stack(3, 'Canada', Canada, 'China', China, 'Mexico', Mexico) as (Country,Total)"
# unPivotDF = pivotDF.select("Product", expr(unpivotExpr)) \
#     .where("Total is not null")
# unPivotDF.show(truncate=False)

In [14]:
# from pyspark.sql.functions import sum

# # Create a Spark session
# spark = SparkSession.builder.appName("CubeExample").getOrCreate()

# # Sample data
# data = [("Animal", "Dog", 10),
#         ("Animal", "Cat", 20),
#         ("Flower", "Jasmine", 30),
#         ("Flower", "Sunflower", 40)]

# # Define the schema
# schema = ["Category", "Subcategory", "Price"]

# # Create a DataFrame
# df = spark.createDataFrame(data, schema=schema)


# df.show()
# df.cube("Category", "Subcategory").agg(sum("Price").alias("TotalValue")).show()


+--------+-----------+-----+
|Category|Subcategory|Price|
+--------+-----------+-----+
|  Animal|        Dog|   10|
|  Animal|        Cat|   20|
|  Flower|    Jasmine|   30|
|  Flower|  Sunflower|   40|
+--------+-----------+-----+

+--------+-----------+----------+
|Category|Subcategory|TotalValue|
+--------+-----------+----------+
|    NULL|        Cat|        20|
|    NULL|        Dog|        10|
|    NULL|       NULL|       100|
|  Animal|        Dog|        10|
|  Animal|       NULL|        30|
|  Animal|        Cat|        20|
|    NULL|    Jasmine|        30|
|  Flower|       NULL|        70|
|  Flower|  Sunflower|        40|
|  Flower|    Jasmine|        30|
|    NULL|  Sunflower|        40|
+--------+-----------+----------+

